<a href="https://colab.research.google.com/github/sheldonkemper/portfolio/blob/main/CAM_DS_301_W8_Activity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Activity 8.2.3 Performing time series forecasting with machine learning and deep learning

## Scenario
The Baltimore Police Department (BPD), situated in Baltimore, USA, created a data set containing crime records from 1 January 2011 to 18 June 2016. Although the data set is preliminary and subject to change, it contains 264,496 rows and 11 columns (BPD, 2024).

For this activity, the data set was adapted to indicate the number of daily crime incidents from 1 January 2011 to 31 December 2015. Therefore, the data set contains 2,143 rows and two columns. You are tasked to analyse the provided data set to determine visible crime trends. You must also provide a clear and precise rationale for your analysis and findings.


## Objective
In this portfolio activity, you will analyse a time series data set to gain insights into crime trends.

You will complete the activity in your Notebook, where you will:

* Load the preprocessed time series data set.
* Split the data set into training and testing sets.
* Choose a suitable machine learning technique (e.g. LightGBM, XGBoost) and train a model on the training data.
* Implement a deep learning model (e.g. LSTM) for time series forecasting and train it on the same data set.
* Compare the performance of the deep learning model with the machine learning model.
* Explore hybrid techniques by combining the results of classical and machine learning models.
* Present the findings, model comparisons, and insights from the forecasting analysis.



## Assessment criteria
By completing this activity, you will be able to provide evidence that you can critically select appropriate strategies to demonstrate expertise in building and refining decision trees and ensemble techniques.


## Activity guidance

* Load the preprocessed time series data set.
* Split the data set into training and testing sets.
* Choose a suitable machine learning technique (e.g., LightGBM, XGBoost) and train a model on the training data.
* Implement a deep learning model (e.g., LSTM) for time series forecasting and train it on the same data set.
* Compare the performance of the deep learning model with the machine learning model.
* Explore hybrid techniques by combining the results of classical and machine learning models.
* Present the findings, model comparisons, and insights from the forecasting analysis.